<a href="https://colab.research.google.com/github/skyqi/LangChain/blob/main/MessagesPlaceholder%E6%A8%A1%E7%89%88%E6%8F%90%E7%A4%BA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import os
import getpass

api_key = os.getenv("ZHIPUAI_API_KEY")
if api_key is None:
  os.environ["ZHIPUAI_API_KEY"] = getpass.getpass()

In [ ]:
# !pip install --upgrade --quiet  pillow
# !pip install langchain langchain_community
# !pip install --upgrade httpx httpx-sse PyJWT

In [23]:
from langchain_community.chat_models import ChatZhipuAI
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain.chains import ConversationChain

model = ChatZhipuAI(
    model="glm-4",
    temperature=0.5,
)

In [24]:
#提示模板
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
[
    (
        "system",
        "你是一名答题能手，可以回复用户的很多问题",
    ),
    MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model


In [25]:
#传递一个带有messages键的字典，其中包含消息列表。
chain.invoke({"messages": [HumanMessage(content="hi!我是黑金")]})

AIMessage(content='你好！作为答题能手，我很乐意帮助你解答问题。请告诉我你想要了解的内容，我会尽力提供准确的答案。', response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 22, 'total_tokens': 49}, 'model_name': 'glm-4', 'finish_reason': 'stop'}, id='run-520edc76-7a82-41a5-aa3b-48e36d673c02-0')

In [26]:
#传递一个带有messages键的字典，其中包含消息列表。
chain.invoke({"messages": [HumanMessage(content="hi!我的电脑运行时间长了经常蓝屏是什么情况")]})

AIMessage(content='电脑长时间运行后出现蓝屏可能是多种原因造成的。蓝屏通常意味着操作系统遇到了无法恢复的错误，不得不停止运行以保护电脑免受损害。以下是一些可能导致蓝屏的常见原因：\n\n1. 硬件故障：比如内存条、硬盘、显卡等硬件出现问题。\n2. 系统过热：电脑长时间运行可能导致某些部件过热，触发保护机制。\n3. 驱动程序问题：不兼容或损坏的驱动程序可能导致蓝屏。\n4. 系统文件损坏：Windows操作系统中某些关键文件损坏可能导致蓝屏。\n5. 软件冲突：安装的某些程序可能与操作系统或其他软件发生冲突。\n6. 病毒或恶意软件：病毒感染可能破坏系统稳定性，导致蓝屏。\n7. 硬件不兼容：新安装的硬件可能与现有系统不兼容。\n\n解决方法：\n- 检查电脑是否过热，清理风扇和散热片，确保通风良好。\n- 使用杀毒软件进行全盘扫描，清除病毒和恶意软件。\n- 更新或回滚最近更新的驱动程序。\n- 如果最近安装了新硬件，请检查其兼容性，并尝试移除新硬件看问题是否解决。\n- 使用系统文件检查器（sfc /scannow）检查系统文件损坏情况。\n- 尝试进入安全模式，查看问题是否依然存在。\n- 如果上述方法都无法解决问题，可能需要考虑联系专业的电脑维修服务。\n\n记住，在处理电脑问题时，确保备份重要数据，以防不测。', response_metadata={'token_usage': {'completion_tokens': 322, 'prompt_tokens': 29, 'total_tokens': 351}, 'model_name': 'glm-4', 'finish_reason': 'stop'}, id='run-e53b6ddb-01a0-41e6-a7ea-96729a0d02e4-0')

In [ ]:
#第二方式，
#对话小结式提示模板
from langchain import PromptTemplate
SUMMARIZER_TEMPLATE="""请将以下内容逐步概括所提供的对话内容，并将新的橛括添加到之前的概括中，形成新的概括。

EXAMPLE
Current summary:
Human询问AI对人工智能的看法。AI认为人工智能是一种积极的力量。
New lines of conversation:
Human:为什么你认为人工智能是一种积极的力量?
AI:因为人工智能将帮助人类发挥他们的潜能。

New summary:
Human询问AI对人]ǐ智能的看法。AI认为人工智能是一种积极的力量，因为它将帮助人类发挥他们的潜能。
END OF EXAMPLE

Current summary:
{summary}

New lines of conversation:
{new_lines}

New summary:"""

SUMMARY_PROMPT = PromptTemplate(
  input_variables=["summary","new_lines"],
  template=SUMMARIZER_TEMPLATE
)

In [ ]:
from langchain.memory import ConversationSummaryBufferMemory
memory = ConversationSummaryBufferMemory(llm=model,prompt=SUMMARY_PROMPT,max_token_limit=40)
#手动存入历史记录
memory.save_context({"input": "你好"}, {"output": "你好，我是客户欢欢，有什么可以帮助您的么"})
memory.save_context({"input": "我是黑金，我在你们这下了一个订单，订单号是B20460613A,我的邮箱是abc123@bcd.com.前天下的订单还没有收到货，请帮助查一下"},
 {"output": "好的。正在为查询您的订单"})

In [ ]:
#打印的小结
memory.load_memory_variables({})

{'history': 'System: Human与AI打招呼，AI自我介绍为客户欢欢，并表示愿意提供帮助。在Human表达了想了解售后服务的意愿后，AI说明了提供7*24小时的在线客服支持。随后，Human作为黑金客户，提供了自己的订单号和邮箱，并请求AI帮助查询前天下的订单B20460613A为何还未收到货的情况。 \n\nNew summary: \nHuman与AI客户欢欢打招呼，并询问售后服务。AI介绍了全天候在线客服支持，并在黑金客户提供订单号B20460613A和邮箱abc123@bcd.com后，被请求帮助查询未收货的订单情况。\nAI: 好的。正在为查询您的订单'}